In [1]:
import argparse
import os

import apache_beam as beam
import tensorflow as tf
from apache_beam.options.pipeline_options import PipelineOptions
import apache_beam.runners.interactive.interactive_beam as ib
import apache_beam.transforms.sql

import beam__common
import fidscs_globals

import data_extractor
import preprocessor

# from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow.keras.preprocessing.image

In [2]:
data_dir = "/tmp/fids-capstone-data/data"

In [3]:
data_extractor.run(max_target_videos=-1, data_dir=data_dir, use_beam=True)

use_beam: True
Found dataset /tmp/fids-capstone-data/data/consultant-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-targetvideo-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-utterance-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-utterance-targetvideo-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-utterance-token-index.csv
Found dataset /tmp/fids-capstone-data/data/ncslgr-corpus-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-index.csv
Found dataset /tmp/fids-capstone-data/data/document-consultant-targetvideo-utterance-token-frame-index.csv
Found dataset /tmp/fids-capstone-data/data/vocabulary-index.csv


In [4]:
preprocessor.run(data_dir=data_dir)

Found train/val dataset /tmp/fids-capstone-data/data/train-assoc.csv
Found train/val dataset /tmp/fids-capstone-data/data/val.csv
Found train/val dataset /tmp/fids-capstone-data/data/train.csv
Found train/val dataset /tmp/fids-capstone-data/data/complete-utterances-train-assoc.csv
Found train/val dataset /tmp/fids-capstone-data/data/complete-utterances-val.csv
Found train/val dataset /tmp/fids-capstone-data/data/complete-utterances-train.csv


In [5]:
options = {
    'project': 'my-project', # change
    'runner': 'InteractiveRunner',
    'direct_num_workers': 0, # 0 is use all available cores
    'direct_running_mode': 'multi_threading', # ['in_memory', 'multi_threading', 'multi_processing'] # 'multi_processing' doesn't seem to work for DirectRunner?
    'streaming': False # set to True if data source is unbounded (e.g. GCP PubSub)
}
pipeline_options = PipelineOptions(flags=[], **options) # easier to pass in options from command-line this way
print(f"PipelineOptions:\n{pipeline_options.get_all_options()}\n")

PipelineOptions:
{'runner': 'InteractiveRunner', 'streaming': False, 'beam_services': {}, 'type_check_strictness': 'DEFAULT_TO_ANY', 'type_check_additional': '', 'pipeline_type_check': True, 'runtime_type_check': False, 'performance_runtime_type_check': False, 'direct_runner_use_stacked_bundle': True, 'direct_runner_bundle_repeat': 0, 'direct_num_workers': 0, 'direct_running_mode': 'multi_threading', 'dataflow_endpoint': 'https://dataflow.googleapis.com', 'project': 'my-project', 'job_name': None, 'staging_location': None, 'temp_location': None, 'region': None, 'service_account_email': None, 'no_auth': False, 'template_location': None, 'labels': None, 'update': False, 'transform_name_mapping': None, 'enable_streaming_engine': False, 'dataflow_kms_key': None, 'flexrs_goal': None, 'hdfs_host': None, 'hdfs_port': None, 'hdfs_user': None, 'hdfs_full_urls': False, 'num_workers': None, 'max_num_workers': None, 'autoscaling_algorithm': None, 'machine_type': None, 'disk_size_gb': None, 'disk_t

In [6]:
fidscs_globals.DATA_ROOT_DIR = data_dir

In [7]:
can_proceed = True

if not data_extractor__common.path_exists(fidscs_globals.DATA_ROOT_DIR) or len(beam__common.list_dir(fidscs_globals.DATA_ROOT_DIR))==0:
    print(f"{fidscs_globals.VALIDATION_FATAL_ERROR_TEXT} data directory does not exist or is empty!")
    can_proceed = False
else:
    fidscs_globals.VIDEO_DIR = os.path.join(fidscs_globals.DATA_ROOT_DIR, 'videos')
    fidscs_globals.STICHED_VIDEO_FRAMES_DIR = os.path.join(fidscs_globals.DATA_ROOT_DIR, 'stitched_video_frames')
    fidscs_globals.CORPUS_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.CORPUS_DS_FNAME)
    fidscs_globals.DOCUMENT_ASL_CONSULTANT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.DOCUMENT_ASL_CONSULTANT_DS_FNAME)
    fidscs_globals.ASL_CONSULTANT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.ASL_CONSULTANT_DS_FNAME)
    fidscs_globals.VIDEO_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_DS_FNAME)
    fidscs_globals.VIDEO_SEGMENT_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_SEGMENT_DS_FNAME)
    fidscs_globals.VIDEO_FRAME_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VIDEO_FRAME_DS_FNAME)
    fidscs_globals.UTTERANCE_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_DS_FNAME)
    fidscs_globals.UTTERANCE_VIDEO_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_VIDEO_DS_FNAME)
    fidscs_globals.UTTERANCE_TOKEN_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_TOKEN_DS_FNAME)
    fidscs_globals.UTTERANCE_TOKEN_FRAME_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.UTTERANCE_TOKEN_FRAME_DS_FNAME)
    fidscs_globals.VOCABULARY_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VOCABULARY_DS_FNAME)
    fidscs_globals.TRAIN_ASSOC_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.TRAIN_FRAME_SEQ_ASSOC_DS_FNAME)
    fidscs_globals.VAL_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.VAL_FRAME_SEQ_DS_FNAME)
    fidscs_globals.TRAIN_DS_PATH = os.path.join(fidscs_globals.DATA_ROOT_DIR, fidscs_globals.TRAIN_FRAME_SEQ_DS_FNAME)

In [8]:
pl = beam.Pipeline(options=pipeline_options)

def train_val_index_schemad_pcoll_rows__to__ordered_tuples(train_val_index_schemad_pcoll_row):
    """
    """
    return (
        # SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX = [
        #     'TokenID',
        #     'CameraPerspective',
        #     'ASLConsultantID',
        #     'TargetVideoFilename',
        #     'UtteranceSequence',
        #     'TokenSequence',
        #     'FrameSequence'
        # ]
        train_val_index_schemad_pcoll_row.TokenID,
        train_val_index_schemad_pcoll_row.CameraPerspective,
        train_val_index_schemad_pcoll_row.ASLConsultantID,
        train_val_index_schemad_pcoll_row.TargetVideoFilename,
        train_val_index_schemad_pcoll_row.UtteranceSequence,
        train_val_index_schemad_pcoll_row.TokenSequence,
        train_val_index_schemad_pcoll_row.FrameSequence
    )

train_frame_sequences__assoc_index_schemad_pcoll = beam__common.pl__1__read_train_frame_sequences__assoc_index_csv(pl)
train_frame_sequences__assoc_index = (
    train_frame_sequences__assoc_index_schemad_pcoll
    | "Beam PL: transform train_frame_sequences__assoc_index_schemad_pcoll rows to ordered tuples (according to schema)" >> beam.Map(train_val_index_schemad_pcoll_rows__to__ordered_tuples)
)

val_frame_sequences_index_schemad_pcoll = beam__common.pl__1__read_val_frame_sequences__index_csv(pl)
val_frame_sequences_index = (
    val_frame_sequences_index_schemad_pcoll
    | "Beam PL: transform val_frame_sequences_index_schemad_pcoll rows to ordered tuples (according to schema)" >> beam.Map(train_val_index_schemad_pcoll_rows__to__ordered_tuples)
)

train_frame_sequences_index_schemad_pcoll = beam__common.pl__1__read_train_frame_sequences_index_csv(pl)
train_frame_sequences_index = (
    train_frame_sequences_index_schemad_pcoll
    | "Beam PL: transform train_frame_sequences_index_schemad_pcoll rows to ordered tuples (according to schema)" >> beam.Map(train_val_index_schemad_pcoll_rows__to__ordered_tuples)
)

In [9]:
def train_val_index__load_frame_img(train_val_index_tpl):
    """
    SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX = [
        'TokenID',
        'CameraPerspective',
        'ASLConsultantID',
        'TargetVideoFilename',
        'UtteranceSequence',
        'TokenSequence',
        'FrameSequence'
    ]
    """
    target_video_frames_dir = os.path.join(
        fidscs_globals.STICHED_VIDEO_FRAMES_DIR, 
        train_val_index_tpl[3].split('.')[0]
    )
    target_video_frame_fname = f"{train_val_index_tpl[6]}.jpg"
    target_video_frame_path = os.path.join(target_video_frames_dir, target_video_frame_fname)
    target_video_frame_img = tf.keras.preprocessing.image.load_img(
        target_video_frame_path, 
        target_size=fidscs_globals.FRAME_IMG_INPUT_SHAPE
    )  # this is a PIL image
    target_video_frame_img_array = tf.keras.preprocessing.image.img_to_array(target_video_frame_img)
    target_video_frame_img_array = target_video_frame_img_array.reshape((1,) + target_video_frame_img_array.shape)
    target_video_frame_img_array /= 255.0 # Rescale by 1/255

    return (
        train_val_index_tpl[0],
        train_val_index_tpl[1],
        train_val_index_tpl[2],
        train_val_index_tpl[3],
        train_val_index_tpl[4],
        train_val_index_tpl[5],
        train_val_index_tpl[6],
        target_video_frame_img_array
    )

In [10]:
def pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(train_val_frame_sequences_index, name_train_val_frame_sequences_index, n_sample_size=50, rand=False):
    """
    """

    sample = (
        train_val_frame_sequences_index
        | f"Beam PL: get random sample of {n_sample_size} frames from {name_train_val_frame_sequences_index}" >> beam.combiners.Sample.FixedSizeGlobally(n_sample_size)
    ) if rand else (
        train_val_frame_sequences_index
        | f"Beam PL: get top {n_sample_size} frames from {name_train_val_frame_sequences_index}" >> beam.combiners.Top.Of(n_sample_size)
    )
    return (
        sample
        | f"Beam PL: flatten sample of {n_sample_size} frames from {name_train_val_frame_sequences_index}" >> beam.FlatMap(lambda lst_tpl: lst_tpl)
        | f"Beam PL: load frame images of {name_train_val_frame_sequences_index}" >> beam.Map(train_val_index__load_frame_img)
    )

train_frame_sequences__assoc_index__sample__with_frame_tensors = pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(
    train_frame_sequences__assoc_index, 
    "train_frame_sequences__assoc_index"
)
val_frame_sequences_index__sample__with_frame_tensors = pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(
    val_frame_sequences_index, 
    "val_frame_sequences_index"
)
train_frame_sequences_index__sample__with_frame_tensors = pl__X__sample_train_val_frame_sequences_index__with_frame_tensors(
    train_frame_sequences_index, 
    "train_frame_sequences_index"
)

In [11]:
# we require this in order to make use of ib.show() (which provides visualization of the pcolls specified) or ib.collect() (which creates a pandas dataframe from a pcoll)
    # but all pcolls we wish to visualize must be created prior to executing the following line
ib.watch(locals())

<p><br>

##### Show base train/val frame datasets

In [12]:
df_train_frame_sequences__assoc_index = ib.collect(train_frame_sequences__assoc_index)

In [13]:
df_train_frame_sequences__assoc_index.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX
df_train_frame_sequences__assoc_index.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_train_frame_sequences__assoc_index.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences__assoc_index.sort_index(inplace=True)

df_train_frame_sequences__assoc_index

FrameSequence
TokenID CameraPerspective ASLConsultantID TargetVideoFilename        UtteranceSequence TokenSequence               
0       0                 2               roadtrip2_1051_small_0.mov 44                0                       5348
                                                                                       0                       5349
                                                                                       0                       5350
                                                                                       0                       5351
                                                                                       0                       5352
...                                                                                                             ...
2397    2                 2               football_1073_small_2.mov  31                5                       3525
                                                                                       5                       3526
                                                                                       5                       3527
                                                                                       5                       3528
                                                                                       5                       3529

[196769 rows x 1 columns]

In [14]:
df_val_frame_sequences_index = ib.collect(val_frame_sequences_index)

In [15]:
df_val_frame_sequences_index.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX
df_val_frame_sequences_index.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_val_frame_sequences_index.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_val_frame_sequences_index.sort_index(inplace=True)

df_val_frame_sequences_index

FrameSequence
TokenID CameraPerspective ASLConsultantID TargetVideoFilename         UtteranceSequence TokenSequence               
0       0                 6               ben_story_441_small_0.mov   33                0                       2437
                                                                                        0                       2438
                                                                                        0                       2439
                                                                                        0                       2440
                                                                                        0                       2441
...                                                                                                              ...
2397    2                 2               lapd_story_1083_small_2.mov 86                8                       6519
                                                                                        8                       6520
                                                                                        8                       6521
                                                                                        8                       6522
                                                                                        8                       6523

[40465 rows x 1 columns]

In [16]:
df_train_frame_sequences_index = ib.collect(train_frame_sequences_index)

In [17]:
df_train_frame_sequences_index.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX
df_train_frame_sequences_index.set_index(fidscs_globals.SCHEMA_PK__TRAIN_OR_VAL_INDEX, inplace=True)
df_train_frame_sequences_index.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences_index.sort_index(inplace=True)

df_train_frame_sequences_index

FrameSequence
TokenID CameraPerspective ASLConsultantID TargetVideoFilename        UtteranceSequence TokenSequence               
0       0                 2               roadtrip2_1051_small_0.mov 44                0                       5348
                                                                                       0                       5349
                                                                                       0                       5350
                                                                                       0                       5351
                                                                                       0                       5352
...                                                                                                             ...
2411    2                 2               boston-la_1088_small_2.mov 70                12                     11323
                                                                                       12                     11324
                                                                                       12                     11325
                                                                                       12                     11326
                                                                                       12                     11327

[250945 rows x 1 columns]

<p><br>

##### Show sample of train/val frame datasets with loaded tensors (to demonstrate functional frame-to-tensor extraction)

In [18]:
df_train_frame_sequences__assoc_index__sample__with_frame_tensors = ib.collect(train_frame_sequences__assoc_index__sample__with_frame_tensors)

In [19]:
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX + ['FrameTensor']
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.set_index(['ASLConsultantID', 'TargetVideoFilename', 'CameraPerspective', 'UtteranceSequence', 'TokenSequence'], inplace=True)
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences__assoc_index__sample__with_frame_tensors.sort_index(inplace=True)

df_train_frame_sequences__assoc_index__sample__with_frame_tensors

TokenID  \
ASLConsultantID TargetVideoFilename       CameraPerspective UtteranceSequence TokenSequence            
2               football_1073_small_0.mov 0                 31                5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                football_1073_small_2.mov 2                 31                5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
                                                                              5                 2397   
5               DSP%2520DeadDog.mov       0                 23                4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
                                                                              4                 2379   
6               ben_story_443_small_0.mov 0                 21                3                 2360   
                ben_story_443_small_1.mov 1                 21                3                 2360   
                                                                              3                 2360   
                                                                              3                 2360   
                                                                              3                 2360   
                ben_story_443_small_2.mov 2                 21                3                 2360   
                                                                              3                 2360   
                                                                              3                 2360   
                                                                              3                 2360   
                ben_story_443_small_3.mov 3                 11                2                 2360   
                                                                              2                 2360   
                                                                              2                 2360   
                                                                              2                 2360   
                                                                              2                 2360   
                                                                              2                 2360   
                                     

In [20]:
df_val_frame_sequences_index__sample__with_frame_tensors = ib.collect(val_frame_sequences_index__sample__with_frame_tensors)

In [21]:
df_val_frame_sequences_index__sample__with_frame_tensors.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX + ['FrameTensor']
df_val_frame_sequences_index__sample__with_frame_tensors.set_index(['ASLConsultantID', 'TargetVideoFilename', 'CameraPerspective', 'UtteranceSequence', 'TokenSequence'], inplace=True)
df_val_frame_sequences_index__sample__with_frame_tensors.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_val_frame_sequences_index__sample__with_frame_tensors.sort_index(inplace=True)

df_val_frame_sequences_index__sample__with_frame_tensors

TokenID  \
ASLConsultantID TargetVideoFilename         CameraPerspective UtteranceSequence TokenSequence            
2               lapd_story_1083_small_0.mov 0                 86                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                lapd_story_1083_small_2.mov 2                 86                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
                                                                                8                 2397   
5               DSP%2520DeadDog.mov         0                 24                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
                                                                                1                 2379   
6               ben_story_443_small_2.mov   2                 11                2                 2360   
                                                                                2                 2360   
                                                                                2                 2360   
                                                                                2                 2360   
                                                                                2                 2360   
                                                                   

In [22]:
df_train_frame_sequences_index__sample__with_frame_tensors = ib.collect(train_frame_sequences_index__sample__with_frame_tensors)

In [23]:
df_train_frame_sequences_index__sample__with_frame_tensors.columns = fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX + ['FrameTensor']
df_train_frame_sequences_index__sample__with_frame_tensors.set_index(['ASLConsultantID', 'TargetVideoFilename', 'CameraPerspective', 'UtteranceSequence', 'TokenSequence'], inplace=True)
df_train_frame_sequences_index__sample__with_frame_tensors.sort_values(axis=0, by=[fidscs_globals.SCHEMA_COL_NAMES__TRAIN_OR_VAL_INDEX[6]], ignore_index=False, inplace=True)
df_train_frame_sequences_index__sample__with_frame_tensors.sort_index(inplace=True)

df_train_frame_sequences_index__sample__with_frame_tensors

TokenID  \
ASLConsultantID TargetVideoFilename        CameraPerspective UtteranceSequence TokenSequence            
1               DSP%2520Introduction.mov   0                 6                 1                 2409   
                                                                               1                 2409   
                                                                               1                 2409   
                                                                               1                 2409   
2               boston-la_1088_small_0.mov 0                 27                15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                             70                12                2411   
                                                                               12                2411   
                                                                               12                2411   
                                                                               12                2411   
                                                                               12                2411   
                                                                               12                2411   
                boston-la_1088_small_2.mov 2                 27                15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   
                                                                               15                2410   